# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [ ]:
# imports

from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
import os

In [ ]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

OLLAMA_BASE_URL = 'http://localhost:11434/v1'

In [ ]:
# set up environment

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

In [ ]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [ ]:
from openai import api_key

system_prompt = """you are a helpful assistant that can explain a technical question in a way that is easy to understand.
                    give a detailed explanation of the question and the logic behind it."""


In [ ]:
# A tool assistant class to wrap assistant variables and functions

class TechnicalAssistant:
    def __init__(self, openai_api_key = None):
        self.openai_client = OpenAI(api_key=openai_api_key)
        self.ollama_client = OpenAI(base_url=OLLAMA_BASE_URL, api_key="ollama")
        self.model_gpt = MODEL_GPT
        self.model_llama = MODEL_LLAMA

    def _get_client_and_model(self, provider):
        #if gpt or open ai in provider
        if "gpt" in provider.lower():
            return self.openai_client, self.model_gpt
        elif "llama" in provider.lower():
            return self.openai_client, self.model_gpt
        else:
            raise ValueError(f"Unsupported provider: {provider}")

    def answer(self, question: str, provider: str="gpt"):
        client, model = self._get_client_and_model(provider)
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
        ]
        stream = client.chat.completions.create(
            model=model,
            messages=messages,
            stream=True
        )
        response = ""
        display_handle = display(Markdown(""), display_id=True)
        for chunk in stream:
            response += chunk.choices[0].delta.content or ''
            update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
# Get gpt-4o-mini to answer, with streaming

assistant = TechnicalAssistant()

assistant.answer(question, "gpt")


In [ ]:
# Get Llama 3.2 to answer

assistant.answer(question, "ollama")